First, I need to try and figure out a way to download data from coinbase.

In [39]:
from secret import api_key, api_secret

from coinbase.wallet.client import Client
from time import sleep

import json
import numpy as np

In [8]:
client = Client(api_key, api_secret)

In [11]:
client.get_spot_price(currency_pair = 'ETH-USD')

<APIObject @ 0x1107d1db0> {
  "amount": "448.57",
  "base": "ETH",
  "currency": "USD"
}

In [15]:
t = 1
act = 10**6
unit = 1
thres = 1
last = 0

while True:
    print(f'timestep: {t}, account: ${act}')
    print(client.get_spot_price(currency_pair = 'ETH-USD'))
    if 
    
    
    t += 1
    sleep(60)

1
{
  "amount": "448.39",
  "base": "ETH",
  "currency": "USD"
}
2
{
  "amount": "448.24",
  "base": "ETH",
  "currency": "USD"
}
3
{
  "amount": "447.41",
  "base": "ETH",
  "currency": "USD"
}


KeyboardInterrupt: 

In [16]:
res = client.get_spot_price(currency_pair = 'ETH-USD')

In [20]:
import json
data = json.loads(str(res))
data['amount']

'448.67'

Now we're going to implement a very simple trading strategy where we assume that there is no transaction costs.  basicallly, you trade against the movements of the currency.

In [27]:
t = 1
act = {'dol': 10**6, 'eth': 0, 'net': 0}
unit = 10
thres = 0.1
last = 0

while True:
    res = client.get_spot_price(currency_pair = 'ETH-USD')
    data = json.loads(str(res))
    
    curr_price = float(data['amount'])
    # buy
    if curr_price + thres < last and act['dol'] > unit * curr_price:
        act['dol'] -= curr_price * unit
        act['eth'] += unit
    
    # sell
    if curr_price - thres > last and act['eth'] > unit:
        act['dol'] += curr_price * unit
        act['eth'] -= unit
        
    # update total net worth
    act['net'] = act['dol'] + act['eth'] * curr_price
    print(f'timestep: {t}, spot price: {curr_price}, account: ${act}')
    last = curr_price
    t += 1
    sleep(60)

timestep: 1, spot price: 452.97, account: ${'dol': 1000000, 'eth': 0, 'net': 1000000.0}
timestep: 2, spot price: 453.56, account: ${'dol': 1000000, 'eth': 0, 'net': 1000000.0}
timestep: 3, spot price: 452.74, account: ${'dol': 995472.6, 'eth': 10, 'net': 1000000.0}
timestep: 4, spot price: 453.17, account: ${'dol': 995472.6, 'eth': 10, 'net': 1000004.2999999999}
timestep: 5, spot price: 453.44, account: ${'dol': 995472.6, 'eth': 10, 'net': 1000007.0}
timestep: 6, spot price: 453.27, account: ${'dol': 990939.9, 'eth': 20, 'net': 1000005.3}
timestep: 7, spot price: 453.41, account: ${'dol': 995474.0, 'eth': 10, 'net': 1000008.1}
timestep: 8, spot price: 453.48, account: ${'dol': 995474.0, 'eth': 10, 'net': 1000008.8}
timestep: 9, spot price: 453.03, account: ${'dol': 990943.7, 'eth': 20, 'net': 1000004.2999999999}
timestep: 10, spot price: 453.03, account: ${'dol': 990943.7, 'eth': 20, 'net': 1000004.2999999999}
timestep: 11, spot price: 453.41, account: ${'dol': 995477.7999999999, 'eth'

KeyboardInterrupt: 

Okay, after that very limited test, it APPEARS that the algorithm is making money.  However, the version used above is incredibly simple.

It doesn't take into account fees at all.  It also isn't fully executing a momentum strategy.  The unit of purchase should actually vary according to how much the price has changed between each time step.  Let's try to add both of those factors now.

First, let's add fee's.  To get results faster, I'll lower the threshold and make things more fun.

In [35]:
t = 1
start_val = 10**6
act = {'dol': start_val, 'eth': 0, 'net': 0}
unit = 100
thres = 0.1 # 10 cent XD
last = 0
fee_pc = 0.0025

while True:
    res = client.get_spot_price(currency_pair = 'ETH-USD')
    data = json.loads(str(res))
    
    curr_price = float(data['amount'])
    coin_num = unit
    order_amount = curr_price * coin_num
    order_tot = order_amount * (1+fee_pc)
    # buy
    if curr_price + thres < last and act['dol'] > order_tot:
        act['dol'] -= order_tot
        act['eth'] += coin_num
    
    # sell
    if curr_price - thres > last and act['eth'] > coin_num:
        act['dol'] += order_amount *(1-fee_pc)
        act['eth'] -= coin_num
        
    # update total net worth
    act['net'] = act['dol'] + act['eth'] * curr_price
    print(f'timestep: {t}, spot price: ${curr_price:,.2f}, account: ${act}')
    print(f"${float(act['net'])-start_val:,.2f}")
    last = curr_price
    t += 1
    sleep(60)

timestep: 1, spot price: $452.70, account: ${'dol': 1000000, 'eth': 0, 'net': 1000000.0}
$0.00
timestep: 2, spot price: $452.70, account: ${'dol': 1000000, 'eth': 0, 'net': 1000000.0}
$0.00
timestep: 3, spot price: $452.70, account: ${'dol': 1000000, 'eth': 0, 'net': 1000000.0}
$0.00
timestep: 4, spot price: $452.70, account: ${'dol': 1000000, 'eth': 0, 'net': 1000000.0}
$0.00
timestep: 5, spot price: $452.70, account: ${'dol': 1000000, 'eth': 0, 'net': 1000000.0}
$0.00


KeyboardInterrupt: 

Ouch! Once we added fees, things went really badly. After 72 timesteps, we were at $-2,014.43.  I need to think more carefully about how to leverage the properites unique to this time series.

Maybe I could also try training models on this data?

In [36]:
t = 1
start_val = 10**6
act = {'dol': start_val, 'eth': 0, 'net': 0}
unit = 100
thres = 0.1 # 10 cent XD
last = 0
fee_pc = 0.0025

# initial purchase
res = client.get_spot_price(currency_pair = 'ETH-USD')
data = json.loads(str(res))
curr_price = float(data['amount'])
coin_num = unit*5
order_amount = curr_price * coin_num
order_tot = order_amount * (1+fee_pc)
act['dol'] -= order_tot
act['eth'] += coin_num

while True:
    res = client.get_spot_price(currency_pair = 'ETH-USD')
    data = json.loads(str(res))
    
    curr_price = float(data['amount'])
    coin_num = unit
    order_amount = curr_price * coin_num
    order_tot = order_amount * (1+fee_pc)
    # buy
    if curr_price + thres < last and act['dol'] > order_tot:
        act['dol'] -= order_tot
        act['eth'] += coin_num
    
    # sell
    if curr_price - thres > last and act['eth'] > coin_num:
        act['dol'] += order_amount *(1-fee_pc)
        act['eth'] -= coin_num
        
    # update total net worth
    act['net'] = act['dol'] + act['eth'] * curr_price
    print(f'timestep: {t}, spot price: ${curr_price:,.2f}, account: ${act}')
    print(f"${float(act['net'])-start_val:,.2f}")
    last = curr_price
    t += 1
    sleep(60)

timestep: 1, spot price: $452.86, account: ${'dol': 818176.7100000001, 'eth': 400, 'net': 999320.7100000001}
$-679.29
timestep: 2, spot price: $452.98, account: ${'dol': 863361.4650000001, 'eth': 300, 'net': 999255.4650000001}
$-744.53
timestep: 3, spot price: $452.83, account: ${'dol': 817965.2575000001, 'eth': 400, 'net': 999097.2575000001}
$-902.74
timestep: 4, spot price: $453.37, account: ${'dol': 863188.915, 'eth': 300, 'net': 999199.915}
$-800.08
timestep: 5, spot price: $453.37, account: ${'dol': 863188.915, 'eth': 300, 'net': 999199.915}
$-800.08
timestep: 6, spot price: $453.07, account: ${'dol': 817768.6475000001, 'eth': 400, 'net': 998996.6475000001}
$-1,003.35


KeyboardInterrupt: 

Hasn't run for very long, but we're already pretty deeply in the red.

One pretty obvious update I haven't made yet: I should make the threshold a percentage of the price rather than a fix number.  In fact, I should make it so that I never trade when the gain is less than the fee.

In [38]:
# run parameters
t = 1
start_val = 10**6
act = {'dol': start_val, 'eth': 0, 'net': 0}
unit = 100
thres = 0.1 # 10 cent XD
last = 0
fee_pc = 0.0025
spread = 0.0025

# initial purchase
res = client.get_spot_price(currency_pair = 'ETH-USD')
data = json.loads(str(res))
curr_price = float(data['amount'])
coin_num = unit*5
order_amount = curr_price * coin_num
order_tot = order_amount * (1+fee_pc)
act['dol'] -= order_tot
act['eth'] += coin_num

while True:
    res = client.get_spot_price(currency_pair = 'ETH-USD')
    data = json.loads(str(res))
    
    curr_price = float(data['amount'])
    coin_num = unit
    order_amount = curr_price * coin_num
    order_tot = order_amount * (1+fee_pc)
    
    thres = curr_price * (fee_pc + spread)
    
    # buy
    if curr_price + thres < last and act['dol'] > order_tot:
        act['dol'] -= order_tot
        act['eth'] += coin_num
    
    # sell
    if curr_price - thres > last and act['eth'] > coin_num:
        act['dol'] += order_amount *(1-fee_pc)
        act['eth'] -= coin_num
        
    # update total net worth
    act['net'] = act['dol'] + act['eth'] * curr_price
    print(f'timestep: {t}, spot price: ${curr_price:,.2f}, account: ${act}')
    print(f"${float(act['net'])-start_val:,.2f}")
    print(f'threshold: ${thres:,.2f}')
    last = curr_price
    t += 1
    sleep(60)

timestep: 1, spot price: $453.86, account: ${'dol': 817775.2100000001, 'eth': 400, 'net': 999319.2100000001}
$-680.79
threshold: $2.27
timestep: 2, spot price: $453.86, account: ${'dol': 817775.2100000001, 'eth': 400, 'net': 999319.2100000001}
$-680.79
threshold: $2.27
timestep: 3, spot price: $453.86, account: ${'dol': 817775.2100000001, 'eth': 400, 'net': 999319.2100000001}
$-680.79
threshold: $2.27
timestep: 4, spot price: $453.28, account: ${'dol': 817775.2100000001, 'eth': 400, 'net': 999087.2100000001}
$-912.79
threshold: $2.27
timestep: 5, spot price: $452.89, account: ${'dol': 817775.2100000001, 'eth': 400, 'net': 998931.2100000001}
$-1,068.79
threshold: $2.26
timestep: 6, spot price: $452.82, account: ${'dol': 817775.2100000001, 'eth': 400, 'net': 998903.2100000001}
$-1,096.79
threshold: $2.26
timestep: 7, spot price: $452.92, account: ${'dol': 817775.2100000001, 'eth': 400, 'net': 998943.2100000001}
$-1,056.79
threshold: $2.26
timestep: 8, spot price: $452.69, account: ${'dol

KeyboardInterrupt: 

I also want to add a moving average component.

In [ ]:
# environment setup
t = 1
start_val = 10**6
act = {'dol': start_val, 'eth': 0, 'net': 0}
unit = 100
last = 0
fee_pc = 0.0025

# run params
thres = 0.1
spread = 0.0001
tdelta = 25


prices =[]

# initial purchase
res = client.get_spot_price(currency_pair = 'ETH-USD')
data = json.loads(str(res))
curr_price = float(data['amount'])
prices.append(curr_price)
coin_num = unit*5
order_amount = curr_price * coin_num
order_tot = order_amount * (1+fee_pc)
#act['dol'] -= order_tot
#act['eth'] += coin_num

while True:
    res = client.get_spot_price(currency_pair = 'ETH-USD')
    data = json.loads(str(res))
    
    curr_price = float(data['amount'])
    prices.append(curr_price)
    coin_num = unit
    order_amount = curr_price * coin_num
    order_tot = order_amount * (1+fee_pc)
    
    thres = curr_price * (fee_pc + spread)
    
    if t < tdelta:
        target_price = curr_price
    else:
        target_price = np.average(prices[t-tdelta:t])
    
    # buy
    if target_price - thres > curr_price and act['dol'] > order_tot:
        act['dol'] -= order_tot
        act['eth'] += coin_num
    
    # sell
    if target_price + thres < curr_price and act['eth'] > coin_num:
        act['dol'] += order_amount *(1-fee_pc)
        act['eth'] -= coin_num
        
    # update total net worth
    act['net'] = act['dol'] + act['eth'] * curr_price
    print(f'timestep: {t}, spot price: ${curr_price:,.2f}, account: ${act}')
    print(f"Profit: ${float(act['net'])-start_val:,.2f}, {(float(act['net'])-start_val)/start_val:,.2f}%")
    print(f'threshold: ${thres:,.2f}')
    print(f'target price: ${target_price:,.2f}')
    print(f'spread: ${curr_price-target_price:,.2f}')
    last = curr_price
    t += 1
    sleep(30)

timestep: 1, spot price: $453.60, account: ${'dol': 1000000, 'eth': 0, 'net': 1000000.0}
Profit: $0.00, 0.00%
threshold: $1.18
target price: $453.60
spread: $0.00


Okay, so at this point, the simulation is going to take a while to return any definitive results.  I really need to get that backtesting data back online.  There are definitely more aspects I can add to my trading model.  I also need to more track of what's going on, like how much I paid for my coins and how much I've paid in fees.

Might also want to print out the spread between the target and spot price.

Can rounding have any effect in this simulation?

Is my algo working right? Looks like it's reacting in the opposite way. Also the comparision should be against curr, not last.

I should really create a way to access the full transaction history too.